# 01a: NumPy & PyTorch Primer

**Week 1, Days 1-2** | Foundations

**Prerequisites**: Basic Python (you're an expert, so ✓)

---

## Learning Objectives

By the end of this notebook, you will be able to:

- [ ] Create and manipulate tensors in NumPy and PyTorch
- [ ] Understand and apply broadcasting rules
- [ ] Use advanced indexing to reshape and slice data
- [ ] Compute gradients automatically with autograd
- [ ] Move tensors between CPU and GPU

---

In [ ]:
# Imports
import numpy as np
import torch
import matplotlib.pyplot as plt

# Config
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

# Check GPU availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.backends.mps.is_available():
    print("MPS (Apple Silicon) available: True")

---

# Part 1: Tensors

## Layer 1: Intuition — What *is* a tensor?

Imagine organizing data in space:

- **A single number** (like temperature: 72°F) is a **scalar** — a point with no extent
- **A list of numbers** (temperatures across a week: [72, 68, 75, ...]) is a **vector** — a line of values
- **A grid of numbers** (temperatures across cities and days) is a **matrix** — a 2D sheet
- **A cube of numbers** (temperatures across cities, days, and times) is a **3D tensor** — a volume

And it keeps going: 4D, 5D, ... nD. A **tensor** is just an n-dimensional array of numbers.

```
0D: scalar       → point          → shape: ()
1D: vector       → line           → shape: (n,)
2D: matrix       → sheet          → shape: (m, n)
3D: 3-tensor     → cube           → shape: (l, m, n)
4D: 4-tensor     → batch of cubes → shape: (b, l, m, n)
```

**Key insight**: The "rank" of a tensor is just how many indices you need to grab a single number.
A matrix needs two indices (row, column). A 3D tensor needs three.

### Why tensors for ML?

Everything in machine learning is a tensor:
- An image is a 3D tensor: (height, width, channels)
- A batch of images is 4D: (batch, height, width, channels)
- A sentence is a 2D tensor: (sequence_length, embedding_dim)
- Attention weights are 4D: (batch, heads, seq_len, seq_len)

## Layer 2: Code + Visualization

In [ ]:
# Creating tensors of different ranks

# 0D: Scalar
scalar_np = np.array(3.14)
scalar_pt = torch.tensor(3.14)
print(f"Scalar: {scalar_np}, shape: {scalar_np.shape}, ndim: {scalar_np.ndim}")

# 1D: Vector
vector_np = np.array([1, 2, 3, 4, 5])
vector_pt = torch.tensor([1, 2, 3, 4, 5])
print(f"Vector: {vector_np}, shape: {vector_np.shape}, ndim: {vector_np.ndim}")

# 2D: Matrix
matrix_np = np.array([[1, 2, 3],
                       [4, 5, 6]])
matrix_pt = torch.tensor([[1, 2, 3],
                          [4, 5, 6]])
print(f"Matrix shape: {matrix_np.shape}, ndim: {matrix_np.ndim}")
print(matrix_np)

# 3D: Batch of matrices
batch_np = np.random.randn(4, 3, 3)  # 4 matrices, each 3×3
batch_pt = torch.randn(4, 3, 3)
print(f"\n3D tensor shape: {batch_np.shape}, ndim: {batch_np.ndim}")

In [ ]:
# Visualizing tensor dimensions

fig, axes = plt.subplots(1, 4, figsize=(14, 3))

# 0D: Scalar as a point
axes[0].scatter([0], [0], s=200, c='blue')
axes[0].set_xlim(-1, 1)
axes[0].set_ylim(-1, 1)
axes[0].set_title('0D: Scalar\nshape: ()', fontsize=12)
axes[0].set_aspect('equal')

# 1D: Vector as a line of points
axes[1].scatter(range(5), [0]*5, s=100, c='blue')
axes[1].set_xlim(-0.5, 4.5)
axes[1].set_ylim(-1, 1)
axes[1].set_title('1D: Vector\nshape: (5,)', fontsize=12)

# 2D: Matrix as a grid
im = axes[2].imshow(np.random.randn(4, 6), cmap='viridis', aspect='auto')
axes[2].set_title('2D: Matrix\nshape: (4, 6)', fontsize=12)

# 3D: Show as stacked matrices
for i in range(3):
    offset = i * 0.3
    rect = plt.Rectangle((offset, offset), 1, 1, fill=True, 
                          facecolor=plt.cm.viridis(i/3), alpha=0.7, 
                          edgecolor='black', linewidth=2)
    axes[3].add_patch(rect)
axes[3].set_xlim(-0.2, 1.8)
axes[3].set_ylim(-0.2, 1.8)
axes[3].set_title('3D: Stack of matrices\nshape: (3, m, n)', fontsize=12)
axes[3].set_aspect('equal')

plt.tight_layout()
plt.show()

## Layer 3: CS Speak

### Terminology

| Term | Meaning |
|------|--------|
| **Rank** (ndim) | Number of dimensions/axes |
| **Shape** | Tuple of sizes along each dimension |
| **Size** (numel) | Total number of elements (product of shape) |
| **dtype** | Data type of elements (float32, int64, etc.) |
| **Stride** | Memory step size to move along each dimension |

### Memory Layout

Tensors are stored as contiguous blocks in memory. A 2×3 matrix:

```
[[a, b, c],     →  Memory: [a, b, c, d, e, f]
 [d, e, f]]         (row-major / C-contiguous)
```

**Strides** tell you how many elements to skip in memory to move one step along each axis:
- Shape (2, 3) with row-major layout → strides (3, 1)
- To go down one row: skip 3 elements
- To go right one column: skip 1 element

### NumPy vs PyTorch

| Feature | NumPy | PyTorch |
|---------|-------|--------|
| Array type | `ndarray` | `Tensor` |
| GPU support | No (need CuPy) | Yes (CUDA, MPS) |
| Autograd | No | Yes |
| Ecosystem | Data science | Deep learning |
| Interop | `.numpy()` from torch | `torch.from_numpy()` |

In [ ]:
# Exploring tensor attributes

t = torch.randn(2, 3, 4)  # Random 2×3×4 tensor

print(f"Shape: {t.shape}")
print(f"Rank (ndim): {t.ndim}")
print(f"Total elements: {t.numel()}")
print(f"Data type: {t.dtype}")
print(f"Device: {t.device}")
print(f"Strides: {t.stride()}")
print(f"Is contiguous: {t.is_contiguous()}")

## Layer 4: Mathematical Formalism

### Definition

A **tensor** of rank $n$ over a field $\mathbb{F}$ (typically $\mathbb{R}$) is a multilinear map:

$$T: \underbrace{V^* \times \cdots \times V^*}_{p \text{ copies}} \times \underbrace{V \times \cdots \times V}_{q \text{ copies}} \to \mathbb{F}$$

where $V$ is a vector space and $V^*$ its dual. This is the physicist's definition.

**For our purposes** (the ML practitioner's definition): A tensor is simply a multidimensional array:

$$T \in \mathbb{R}^{d_1 \times d_2 \times \cdots \times d_n}$$

with elements accessed by $n$ indices:

$$T_{i_1, i_2, \ldots, i_n} \in \mathbb{R}$$

### Notation

- Scalars: lowercase ($x, y, \alpha$)
- Vectors: bold lowercase ($\mathbf{x}, \mathbf{v}$) or with arrow ($\vec{x}$)
- Matrices: bold uppercase ($\mathbf{A}, \mathbf{W}$)
- General tensors: calligraphic ($\mathcal{T}, \mathcal{X}$) or bold with rank annotation ($\mathbf{T}^{(n)}$)

### Shape Algebra

For $\mathbf{A} \in \mathbb{R}^{m \times n}$ and $\mathbf{B} \in \mathbb{R}^{n \times p}$:

$$\mathbf{A}\mathbf{B} \in \mathbb{R}^{m \times p}$$

The inner dimensions must match; the outer dimensions form the result shape.

---

# Part 2: Broadcasting

## Layer 1: Intuition — Stretching to fit

Imagine you have a photo (a 3D tensor: height × width × RGB) and you want to brighten it. 
Brightness is a single number, but somehow `photo + 0.1` just works. How?

**Broadcasting** automatically "stretches" smaller arrays to match larger ones, without actually copying data.

Think of it like a rubber stamp. If you have:
- A 4×4 grid of values
- A single row of 4 values

Broadcasting "stamps" that row across all 4 rows of the grid:

```
Grid (4×4):     Row (1×4):      Result:
[a a a a]       [x y z w]  →    [a+x a+y a+z a+w]
[b b b b]   +                   [b+x b+y b+z b+w]
[c c c c]                       [c+x c+y c+z c+w]
[d d d d]                       [d+x d+y d+z d+w]
```

## Layer 2: Code + Visualization

In [ ]:
# Broadcasting in action

# Scalar + matrix: scalar broadcasts everywhere
matrix = np.array([[1, 2, 3],
                   [4, 5, 6]])
print("Matrix + 10:")
print(matrix + 10)
print()

# Row vector + matrix: row broadcasts down
row = np.array([100, 200, 300])
print(f"Matrix shape: {matrix.shape}, Row shape: {row.shape}")
print("Matrix + row:")
print(matrix + row)
print()

# Column vector + matrix: column broadcasts across
col = np.array([[10],
                [20]])
print(f"Matrix shape: {matrix.shape}, Column shape: {col.shape}")
print("Matrix + column:")
print(matrix + col)

In [ ]:
# Visualizing broadcasting

fig, axes = plt.subplots(2, 4, figsize=(14, 6))

# Example 1: Matrix + scalar
A = np.array([[1, 2], [3, 4]])
b = 10

axes[0, 0].imshow(A, cmap='Blues', vmin=0, vmax=15)
axes[0, 0].set_title('A (2×2)')
for i in range(2):
    for j in range(2):
        axes[0, 0].text(j, i, f'{A[i,j]}', ha='center', va='center', fontsize=14)

axes[0, 1].text(0.5, 0.5, '+', fontsize=30, ha='center', va='center')
axes[0, 1].axis('off')

axes[0, 2].imshow([[b]], cmap='Oranges', vmin=0, vmax=15)
axes[0, 2].set_title('Scalar: 10')
axes[0, 2].text(0, 0, f'{b}', ha='center', va='center', fontsize=14)

result1 = A + b
axes[0, 3].imshow(result1, cmap='Greens', vmin=0, vmax=15)
axes[0, 3].set_title('Result (2×2)')
for i in range(2):
    for j in range(2):
        axes[0, 3].text(j, i, f'{result1[i,j]}', ha='center', va='center', fontsize=14)

# Example 2: Matrix + row
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
row = np.array([100, 200, 300])

axes[1, 0].imshow(A, cmap='Blues')
axes[1, 0].set_title('A (3×3)')
for i in range(3):
    for j in range(3):
        axes[1, 0].text(j, i, f'{A[i,j]}', ha='center', va='center', fontsize=12)

axes[1, 1].text(0.5, 0.5, '+', fontsize=30, ha='center', va='center')
axes[1, 1].axis('off')

axes[1, 2].imshow(row.reshape(1, -1), cmap='Oranges', aspect='auto')
axes[1, 2].set_title('Row (1×3)')
for j in range(3):
    axes[1, 2].text(j, 0, f'{row[j]}', ha='center', va='center', fontsize=12)

result2 = A + row
axes[1, 3].imshow(result2, cmap='Greens')
axes[1, 3].set_title('Result (3×3)')
for i in range(3):
    for j in range(3):
        axes[1, 3].text(j, i, f'{result2[i,j]}', ha='center', va='center', fontsize=10)

for ax in axes.flat:
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
plt.show()

## Layer 3: CS Speak — The Broadcasting Rules

NumPy/PyTorch broadcasting follows these rules:

1. **Right-align shapes** and compare dimensions from the right
2. **Dimensions are compatible** if they're equal OR one of them is 1
3. **Missing dimensions** (on the left) are treated as 1

```
Shape A:     (8, 1, 6, 1)
Shape B:        (7, 1, 5)
                --------
Right-align: (8, 1, 6, 1)
             (1, 7, 1, 5)  ← implicit 1 on left
                --------
Result:      (8, 7, 6, 5)  ← max of each dimension
```

**Common patterns**:

| Operation | Shape A | Shape B | Result | Use case |
|-----------|---------|---------|--------|----------|
| Scalar op | (m, n) | () | (m, n) | Add constant |
| Row-wise | (m, n) | (n,) | (m, n) | Mean subtraction |
| Col-wise | (m, n) | (m, 1) | (m, n) | Normalize by row sum |
| Outer product | (m, 1) | (n,) | (m, n) | All pairs |

**Efficiency note**: Broadcasting is memory-efficient. No data is actually copied—the computation just "pretends" the smaller array is repeated.

In [ ]:
# Broadcasting rule demonstration

def check_broadcast(shape_a, shape_b):
    """Check if two shapes can be broadcast together."""
    a = np.zeros(shape_a)
    b = np.zeros(shape_b)
    try:
        result = a + b
        return f"{shape_a} + {shape_b} → {result.shape}"
    except ValueError as e:
        return f"{shape_a} + {shape_b} → ERROR: {e}"

# These work
print(check_broadcast((3, 4), (4,)))       # Row broadcast
print(check_broadcast((3, 4), (3, 1)))     # Column broadcast
print(check_broadcast((3, 4), (1, 4)))     # Explicit row broadcast
print(check_broadcast((3, 1, 4), (1, 5, 4)))  # Middle dimension
print()

# This fails
print(check_broadcast((3, 4), (3,)))       # Incompatible!

## Layer 4: Mathematical Formalism

Broadcasting can be formalized as implicit replication along singleton dimensions.

Let $\mathbf{A} \in \mathbb{R}^{m \times n}$ and $\mathbf{b} \in \mathbb{R}^{n}$ (a row vector).

The operation $\mathbf{A} + \mathbf{b}$ is mathematically equivalent to:

$$[\mathbf{A} + \mathbf{b}]_{ij} = A_{ij} + b_j$$

Or equivalently, using the all-ones vector $\mathbf{1}_m \in \mathbb{R}^m$:

$$\mathbf{A} + \mathbf{b} = \mathbf{A} + \mathbf{1}_m \mathbf{b}^T$$

where $\mathbf{1}_m \mathbf{b}^T$ is an $m \times n$ matrix with $\mathbf{b}$ repeated in each row.

**Outer product via broadcasting**:

For $\mathbf{u} \in \mathbb{R}^m$ and $\mathbf{v} \in \mathbb{R}^n$:

$$\mathbf{u} \mathbf{v}^T = \mathbf{u}_{:, \text{None}} \cdot \mathbf{v}_{\text{None}, :}$$

where subscript `None` denotes inserting a singleton dimension (NumPy: `np.newaxis`).

In [ ]:
# Outer product via broadcasting

u = np.array([1, 2, 3, 4])  # Shape (4,)
v = np.array([10, 20, 30])  # Shape (3,)

# Method 1: Explicit outer product
outer1 = np.outer(u, v)
print("np.outer(u, v):")
print(outer1)
print()

# Method 2: Broadcasting
outer2 = u[:, np.newaxis] * v[np.newaxis, :]  # (4, 1) * (1, 3) → (4, 3)
print("Broadcasting (4,1) * (1,3):")
print(outer2)
print()

# Method 3: Even simpler broadcasting (numpy infers)
outer3 = u[:, None] * v  # (4, 1) * (3,) → (4, 3)
print("Same result:", np.allclose(outer1, outer2, outer3))

---

# Part 3: Indexing

## Layer 1: Intuition — Slicing and dicing

Think of a tensor as a multidimensional spreadsheet. Indexing lets you:

- **Select cells**: `A[2, 3]` — grab one element
- **Select rows/columns**: `A[2, :]` — grab row 2
- **Select ranges**: `A[1:4, 0:2]` — grab a submatrix
- **Select with conditions**: `A[A > 5]` — grab all elements > 5
- **Fancy indexing**: `A[[0, 2, 4], :]` — grab rows 0, 2, 4

**Key insight**: Python indexing is 0-based and uses `[start:stop:step]` where:
- `start` is inclusive
- `stop` is exclusive
- Negatives count from the end

## Layer 2: Code + Visualization

In [ ]:
# Create a demo matrix
A = np.arange(20).reshape(4, 5)
print("Original matrix A:")
print(A)
print()

In [ ]:
# Basic indexing
print("Single element A[1, 2]:", A[1, 2])  # Row 1, Col 2
print()

print("Row 2: A[2, :]")
print(A[2, :])
print()

print("Column 3: A[:, 3]")
print(A[:, 3])
print()

print("Submatrix A[1:3, 2:5]:")
print(A[1:3, 2:5])

In [ ]:
# Advanced indexing: negative indices, steps, fancy indexing

print("Last row: A[-1, :]")
print(A[-1, :])
print()

print("Every other column: A[:, ::2]")
print(A[:, ::2])
print()

print("Reversed rows: A[::-1, :]")
print(A[::-1, :])
print()

# Fancy indexing: select specific rows
print("Rows 0 and 3: A[[0, 3], :]")
print(A[[0, 3], :])

In [ ]:
# Boolean indexing

print("Original:")
print(A)
print()

# Boolean mask
mask = A > 10
print("Mask (A > 10):")
print(mask)
print()

# Apply mask — returns 1D array of matching elements
print("A[A > 10]:", A[A > 10])
print()

# Modify in-place using boolean indexing
A_copy = A.copy()
A_copy[A_copy > 10] = -1
print("After A[A > 10] = -1:")
print(A_copy)

In [ ]:
# Visualize different indexing operations

A = np.arange(20).reshape(4, 5)

fig, axes = plt.subplots(2, 3, figsize=(12, 8))

def show_selection(ax, title, selected_mask):
    """Highlight selected elements in blue."""
    display = np.zeros_like(A, dtype=float)
    display[selected_mask] = 1
    ax.imshow(display, cmap='Blues', vmin=0, vmax=1)
    for i in range(4):
        for j in range(5):
            color = 'white' if selected_mask[i, j] else 'black'
            ax.text(j, i, f'{A[i,j]}', ha='center', va='center', 
                   fontsize=12, color=color, fontweight='bold')
    ax.set_title(title, fontsize=11)
    ax.set_xticks(range(5))
    ax.set_yticks(range(4))

# Different selections
mask1 = np.zeros((4, 5), dtype=bool); mask1[1, 2] = True
show_selection(axes[0, 0], 'A[1, 2] — Single element', mask1)

mask2 = np.zeros((4, 5), dtype=bool); mask2[2, :] = True
show_selection(axes[0, 1], 'A[2, :] — Row 2', mask2)

mask3 = np.zeros((4, 5), dtype=bool); mask3[:, 3] = True
show_selection(axes[0, 2], 'A[:, 3] — Column 3', mask3)

mask4 = np.zeros((4, 5), dtype=bool); mask4[1:3, 2:5] = True
show_selection(axes[1, 0], 'A[1:3, 2:5] — Submatrix', mask4)

mask5 = np.zeros((4, 5), dtype=bool); mask5[::2, ::2] = True
show_selection(axes[1, 1], 'A[::2, ::2] — Every other', mask5)

mask6 = A > 10
show_selection(axes[1, 2], 'A[A > 10] — Boolean mask', mask6)

plt.tight_layout()
plt.show()

## Layer 3: CS Speak

### Views vs. Copies

**Critical distinction**:
- **Basic slicing** (`A[1:3, :]`) returns a **view** — shares memory with original
- **Fancy indexing** (`A[[1, 2], :]`) returns a **copy** — independent memory

```python
A = np.array([[1, 2], [3, 4]])

view = A[0, :]      # View: modifying view changes A
view[0] = 99        # A is now [[99, 2], [3, 4]]

copy = A[[0], :]    # Copy: independent
copy[0, 0] = 100    # A is unchanged
```

**Why it matters**: Views are memory-efficient but can cause subtle bugs if you modify data unexpectedly.

### Time Complexity

| Operation | Complexity | Returns |
|-----------|------------|--------|
| Basic slice `A[i:j]` | O(1) | View |
| Boolean index `A[mask]` | O(n) | Copy |
| Fancy index `A[indices]` | O(k) | Copy |

### PyTorch Differences

In PyTorch, basic indexing also returns views, but there are additional considerations for autograd:

```python
x = torch.tensor([1., 2., 3.], requires_grad=True)
y = x[0]  # View, gradient flows through
```

In [ ]:
# Views vs copies demonstration

A = np.arange(6).reshape(2, 3)
print("Original A:")
print(A)
print()

# Basic slice → view
view = A[0, :]
print(f"view = A[0, :]: {view}")
print(f"Shares memory: {np.shares_memory(A, view)}")

view[0] = 99
print(f"After view[0] = 99, A becomes:")
print(A)
print()

# Reset
A = np.arange(6).reshape(2, 3)

# Fancy index → copy
copy = A[[0], :]
print(f"copy = A[[0], :]: {copy}")
print(f"Shares memory: {np.shares_memory(A, copy)}")

copy[0, 0] = 99
print(f"After copy[0, 0] = 99, A is unchanged:")
print(A)

## Layer 4: Mathematical Formalism

### Slice Notation

For $\mathbf{A} \in \mathbb{R}^{m \times n}$, the slice $\mathbf{A}_{i_1:i_2, j_1:j_2}$ extracts:

$$\mathbf{B} \in \mathbb{R}^{(i_2-i_1) \times (j_2-j_1)}$$

where $B_{kl} = A_{(i_1+k), (j_1+l)}$ for $k \in [0, i_2-i_1)$ and $l \in [0, j_2-j_1)$.

### Boolean Indexing as Projection

Boolean indexing with mask $\mathbf{m} \in \{0, 1\}^n$ on vector $\mathbf{x} \in \mathbb{R}^n$ is:

$$\mathbf{x}[\mathbf{m}] = \mathbf{P}_\mathbf{m} \mathbf{x}$$

where $\mathbf{P}_\mathbf{m} \in \mathbb{R}^{k \times n}$ is a selection matrix with $k = \sum_i m_i$ rows, each containing a single 1 in the position of a True value in $\mathbf{m}$.

### Diagonal Extraction

For $\mathbf{A} \in \mathbb{R}^{n \times n}$, the diagonal is:

$$\text{diag}(\mathbf{A}) = [A_{00}, A_{11}, \ldots, A_{n-1,n-1}]^T$$

Equivalently: $[\text{diag}(\mathbf{A})]_i = A_{ii}$

---

# Part 4: Autograd

## Layer 1: Intuition — Automatic calculus

Imagine you're hiking on a foggy mountain and you want to reach the valley (lowest point). You can't see far, but you can feel which way is downhill under your feet. That's what **gradients** tell us: the direction of steepest change.

Computing gradients by hand is tedious and error-prone. **Autograd** does it automatically by:

1. **Recording** every operation you do (building a computation graph)
2. **Replaying** backwards to compute gradients via chain rule

Think of it like a tape recorder:
- Forward: Record `x → x² → x² + 3x → ...`
- Backward: Replay in reverse, computing derivatives at each step

**Key insight**: You never write derivative code. Just write the forward computation, and PyTorch figures out the backward pass.

## Layer 2: Code + Visualization

In [ ]:
# Simple autograd example

# Create a tensor and tell PyTorch to track gradients
x = torch.tensor(2.0, requires_grad=True)

# Do some computation
y = x**2 + 3*x + 1

print(f"x = {x.item()}")
print(f"y = x² + 3x + 1 = {y.item()}")

# Compute gradient: dy/dx
y.backward()

print(f"dy/dx = 2x + 3 = {x.grad.item()}")
print(f"(At x=2: 2*2 + 3 = 7)")

In [ ]:
# Multivariate gradients

x = torch.tensor(1.0, requires_grad=True)
y = torch.tensor(2.0, requires_grad=True)

# f(x, y) = x²y + xy²
f = x**2 * y + x * y**2

print(f"f(1, 2) = 1²·2 + 1·2² = {f.item()}")

# Compute gradients
f.backward()

print(f"∂f/∂x = 2xy + y² = 2·1·2 + 2² = {x.grad.item()}")
print(f"∂f/∂y = x² + 2xy = 1² + 2·1·2 = {y.grad.item()}")

In [ ]:
# Visualize gradient as direction of steepest ascent

# Create a function: f(x, y) = sin(x) + cos(y)
def f(x, y):
    return torch.sin(x) + torch.cos(y)

# Compute gradient at several points
points = [(-1.0, 0.5), (0.5, 1.0), (1.5, -0.5), (-0.5, -1.0)]
gradients = []

for px, py in points:
    x = torch.tensor(px, requires_grad=True)
    y = torch.tensor(py, requires_grad=True)
    z = f(x, y)
    z.backward()
    gradients.append((x.grad.item(), y.grad.item()))

# Plot
fig, ax = plt.subplots(figsize=(10, 8))

# Surface contours
xx = np.linspace(-2, 2, 100)
yy = np.linspace(-2, 2, 100)
XX, YY = np.meshgrid(xx, yy)
ZZ = np.sin(XX) + np.cos(YY)

contour = ax.contourf(XX, YY, ZZ, levels=20, cmap='viridis', alpha=0.8)
plt.colorbar(contour, label='f(x, y)')

# Gradient arrows (pointing uphill)
for (px, py), (gx, gy) in zip(points, gradients):
    ax.scatter(px, py, color='red', s=100, zorder=5)
    ax.quiver(px, py, gx, gy, color='red', scale=5, width=0.01,
              label='Gradient (uphill)' if px == points[0][0] else '')
    ax.quiver(px, py, -gx, -gy, color='white', scale=5, width=0.01,
              label='Negative gradient (downhill)' if px == points[0][0] else '')

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Gradients point in direction of steepest ascent\n(White arrows show descent direction)')
ax.legend()
plt.show()

In [ ]:
# The computation graph

x = torch.tensor(2.0, requires_grad=True)

# Each operation creates a node in the graph
a = x * x      # a = x²
b = 3 * x      # b = 3x  
c = a + b      # c = x² + 3x
y = c + 1      # y = x² + 3x + 1

# Print the graph structure
print(f"y = {y}")
print(f"y.grad_fn = {y.grad_fn}")
print(f"Graph: y ← {y.grad_fn.next_functions}")

# Backward pass traverses this graph
y.backward()
print(f"\ndy/dx = {x.grad}")

## Layer 3: CS Speak

### Computation Graphs

PyTorch builds a **Directed Acyclic Graph (DAG)** during forward computation:
- **Nodes**: Tensors
- **Edges**: Operations (functions)

The `grad_fn` attribute stores the backward function for each tensor.

### Backward Pass Algorithm

```python
def backward(loss):
    # Initialize: gradient of loss w.r.t. itself is 1
    grad_outputs = {loss: 1.0}
    
    # Traverse graph in reverse topological order
    for node in reverse_topological_sort(graph):
        grad_output = grad_outputs[node]
        
        # Apply chain rule via grad_fn
        for input_node, grad_input in node.grad_fn(grad_output):
            grad_outputs[input_node] += grad_input  # Accumulate
    
    # Store leaf gradients in .grad
    for leaf in leaves:
        leaf.grad = grad_outputs[leaf]
```

### Key Concepts

| Term | Meaning |
|------|--------|
| `requires_grad=True` | Tensor participates in gradient computation |
| `grad_fn` | Function to compute gradient (set by operations) |
| `backward()` | Compute all gradients via backpropagation |
| `.grad` | Accumulated gradient (on leaf tensors) |
| `torch.no_grad()` | Context manager to disable gradient tracking |
| `detach()` | Create tensor that doesn't track gradients |

### Memory Considerations

- The computation graph is stored in memory during forward pass
- `backward()` frees the graph by default (set `retain_graph=True` to keep)
- Use `torch.no_grad()` for inference to save memory

In [ ]:
# Important autograd patterns

# 1. Gradients accumulate! Always zero them.
x = torch.tensor(2.0, requires_grad=True)
for i in range(3):
    y = x * x
    y.backward()
    print(f"Iteration {i}: x.grad = {x.grad}")

print("\n(Notice gradients accumulate — this is a feature, not a bug!)")
print("In training loops, always call optimizer.zero_grad()\n")

# 2. Zero gradients manually
x = torch.tensor(2.0, requires_grad=True)
for i in range(3):
    if x.grad is not None:
        x.grad.zero_()
    y = x * x
    y.backward()
    print(f"Iteration {i} (after zeroing): x.grad = {x.grad}")

In [ ]:
# 3. no_grad() for inference

x = torch.tensor(2.0, requires_grad=True)

# With gradient tracking (training)
y = x * x
print(f"y requires grad: {y.requires_grad}")
print(f"y has grad_fn: {y.grad_fn is not None}")

# Without gradient tracking (inference)
with torch.no_grad():
    y_no_grad = x * x
    print(f"\ny_no_grad requires grad: {y_no_grad.requires_grad}")
    print(f"y_no_grad has grad_fn: {y_no_grad.grad_fn is not None}")

## Layer 4: Mathematical Formalism

### Chain Rule

For composed functions $y = f(g(x))$:

$$\frac{dy}{dx} = \frac{dy}{dg} \cdot \frac{dg}{dx}$$

For a computation graph with $y = f_n(f_{n-1}(\cdots f_1(x)))$:

$$\frac{dy}{dx} = \frac{\partial f_n}{\partial f_{n-1}} \cdot \frac{\partial f_{n-1}}{\partial f_{n-2}} \cdots \frac{\partial f_1}{\partial x}$$

### Multivariate Chain Rule

For $y = f(\mathbf{u})$ where $\mathbf{u} = g(\mathbf{x})$:

$$\frac{\partial y}{\partial x_i} = \sum_j \frac{\partial y}{\partial u_j} \cdot \frac{\partial u_j}{\partial x_i}$$

Or in matrix form:

$$\nabla_\mathbf{x} y = \mathbf{J}_g^T \nabla_\mathbf{u} y$$

where $\mathbf{J}_g$ is the Jacobian matrix of $g$.

### Gradient of Scalar Loss

In deep learning, we typically have a scalar loss $L = L(\mathbf{\theta})$. The gradient is:

$$\nabla_\theta L = \left[ \frac{\partial L}{\partial \theta_1}, \frac{\partial L}{\partial \theta_2}, \ldots, \frac{\partial L}{\partial \theta_n} \right]^T$$

This is what `loss.backward()` computes and stores in `param.grad` for each parameter.

---

# Part 5: GPU Basics

## Layer 1: Intuition — Parallel workers

Your CPU is like a brilliant professor who can do one thing at a time, very well.

Your GPU is like a classroom of 1000 students — each can only do simple math, but they all work simultaneously.

For matrix operations (where we do the same thing to many numbers), the GPU wins:
- Multiply two 1000×1000 matrices: ~1 billion operations
- CPU: Do them one-by-one (slow)
- GPU: Do thousands in parallel (fast)

**Key insight**: GPUs are fast for parallel operations (matrix math), but slow for sequential logic and data transfer.

## Layer 2: Code + Visualization

In [ ]:
# Check available devices

print("Available devices:")
print(f"  CPU: Always available")
print(f"  CUDA (NVIDIA): {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"    Device name: {torch.cuda.get_device_name(0)}")
print(f"  MPS (Apple Silicon): {torch.backends.mps.is_available()}")

# Select best available device
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

print(f"\nUsing device: {device}")

In [ ]:
# Moving tensors between devices

# Create tensor on CPU (default)
x_cpu = torch.randn(3, 3)
print(f"x_cpu device: {x_cpu.device}")

# Move to GPU/MPS
x_gpu = x_cpu.to(device)
print(f"x_gpu device: {x_gpu.device}")

# Move back to CPU (needed for numpy, plotting, etc.)
x_back = x_gpu.cpu()
print(f"x_back device: {x_back.device}")

# Create directly on device
y = torch.randn(3, 3, device=device)
print(f"y device (created directly): {y.device}")

In [ ]:
# Benchmark: CPU vs GPU
import time

sizes = [100, 500, 1000, 2000]
cpu_times = []
gpu_times = []

for size in sizes:
    # CPU timing
    a_cpu = torch.randn(size, size)
    b_cpu = torch.randn(size, size)
    
    start = time.time()
    for _ in range(10):
        c_cpu = a_cpu @ b_cpu
    cpu_times.append((time.time() - start) / 10)
    
    # GPU timing (if available)
    if device.type != 'cpu':
        a_gpu = a_cpu.to(device)
        b_gpu = b_cpu.to(device)
        
        # Warmup
        _ = a_gpu @ b_gpu
        if device.type == 'cuda':
            torch.cuda.synchronize()
        
        start = time.time()
        for _ in range(10):
            c_gpu = a_gpu @ b_gpu
        if device.type == 'cuda':
            torch.cuda.synchronize()
        gpu_times.append((time.time() - start) / 10)
    else:
        gpu_times.append(None)

# Print results
print(f"{'Size':>6} | {'CPU (ms)':>10} | {'GPU (ms)':>10} | {'Speedup':>8}")
print("-" * 45)
for size, cpu_t, gpu_t in zip(sizes, cpu_times, gpu_times):
    if gpu_t:
        speedup = cpu_t / gpu_t
        print(f"{size:>6} | {cpu_t*1000:>10.2f} | {gpu_t*1000:>10.2f} | {speedup:>7.1f}x")
    else:
        print(f"{size:>6} | {cpu_t*1000:>10.2f} | {'N/A':>10} | {'N/A':>8}")

## Layer 3: CS Speak

### Device Management

```python
# Recommended pattern for device-agnostic code
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move model and data to same device
model = MyModel().to(device)
data = data.to(device)
output = model(data)
```

### Common Pitfalls

1. **Device mismatch**: Operations require tensors on the same device
   ```python
   x_cpu = torch.randn(3)
   y_gpu = torch.randn(3, device='cuda')
   z = x_cpu + y_gpu  # ERROR!
   ```

2. **Forgetting to sync**: GPU operations are asynchronous
   ```python
   torch.cuda.synchronize()  # Wait for GPU to finish
   ```

3. **Data transfer overhead**: Moving data to GPU is slow
   - Batch data transfers
   - Keep data on GPU during training

### Memory Management

```python
# Check GPU memory
torch.cuda.memory_allocated()  # Currently allocated
torch.cuda.memory_reserved()   # Reserved by caching allocator

# Free cached memory
torch.cuda.empty_cache()
```

## Layer 4: Mathematical Formalism

### Why GPUs Excel at Matrix Operations

Matrix multiplication $\mathbf{C} = \mathbf{A}\mathbf{B}$ for $\mathbf{A} \in \mathbb{R}^{m \times k}$, $\mathbf{B} \in \mathbb{R}^{k \times n}$:

$$C_{ij} = \sum_{l=1}^{k} A_{il} B_{lj}$$

This requires $m \cdot n \cdot k$ multiply-add operations.

**Key observation**: Each $C_{ij}$ can be computed **independently**. This is "embarrassingly parallel."

- **CPU**: Compute $C_{ij}$ entries sequentially
- **GPU**: Compute thousands of $C_{ij}$ entries simultaneously

### Computational Intensity

The ratio of compute to memory access determines GPU suitability:

$$\text{Arithmetic Intensity} = \frac{\text{FLOPs}}{\text{Bytes transferred}}$$

For matrix multiplication: $O(n^3)$ FLOPs, $O(n^2)$ memory → High intensity → GPU-friendly

For element-wise operations: $O(n)$ FLOPs, $O(n)$ memory → Low intensity → Often CPU-bound

---

# Exercises

Now it's your turn. Complete these exercises to solidify your understanding.

## Exercise 1: Tensor Surgery

Create a 10×10 matrix filled with ones, then set the diagonal to zeros using indexing (not loops).

In [ ]:
# Exercise 1: Tensor Surgery

def diagonal_to_zero(n):
    """
    Create an n×n matrix of ones with zeros on the diagonal.
    
    Args:
        n: Size of the square matrix
    
    Returns:
        n×n numpy array with 1s everywhere except 0s on diagonal
    """
    # TODO: Create matrix of ones
    # TODO: Set diagonal to zeros (hint: np.diag_indices or np.fill_diagonal)
    pass

# Test
result = diagonal_to_zero(10)
print(result)
print(f"\nDiagonal values: {np.diag(result)}")
print(f"Sum of matrix (should be 90): {result.sum()}")

## Exercise 2: Broadcasting Challenge

Normalize each row of a matrix to sum to 1 (i.e., convert each row to a probability distribution).

**Hint**: Use `keepdims=True` in your sum to maintain shape for broadcasting.

In [ ]:
# Exercise 2: Broadcasting Challenge

def normalize_rows(matrix):
    """
    Normalize each row to sum to 1.
    
    Args:
        matrix: 2D numpy array
    
    Returns:
        Normalized matrix where each row sums to 1
    """
    # TODO: Compute row sums with keepdims=True
    # TODO: Divide matrix by row sums (broadcasting!)
    pass

# Test
test_matrix = np.array([
    [1, 2, 3, 4],
    [10, 20, 30, 40],
    [5, 5, 5, 5]
])

normalized = normalize_rows(test_matrix)
print("Original:")
print(test_matrix)
print("\nNormalized (each row sums to 1):")
print(normalized)
print(f"\nRow sums: {normalized.sum(axis=1)}")

## Exercise 3: Gradient Computation

Compute the gradient of $f(x, y) = x^2 y + \sin(xy)$ at the point $(1, \pi)$ using PyTorch autograd.

**Expected answer**:
- $\frac{\partial f}{\partial x} = 2xy + y\cos(xy)$ → at $(1, \pi)$: $2\pi + \pi\cos(\pi) = 2\pi - \pi = \pi \approx 3.14$
- $\frac{\partial f}{\partial y} = x^2 + x\cos(xy)$ → at $(1, \pi)$: $1 + \cos(\pi) = 1 - 1 = 0$

In [ ]:
# Exercise 3: Gradient Computation

def compute_gradient():
    """
    Compute gradient of f(x,y) = x²y + sin(xy) at (1, π).
    
    Returns:
        Tuple of (df/dx, df/dy) at (1, π)
    """
    # TODO: Create tensors for x=1 and y=π with requires_grad=True
    # TODO: Compute f(x, y)
    # TODO: Call backward()
    # TODO: Return gradients
    pass

# Test
grad_x, grad_y = compute_gradient()
print(f"∂f/∂x at (1, π) = {grad_x:.4f}")
print(f"∂f/∂y at (1, π) = {grad_y:.4f}")
print(f"\nExpected: ∂f/∂x ≈ π ≈ {np.pi:.4f}, ∂f/∂y ≈ 0")

---

# Why This Matters

Everything we've covered is foundational for the rest of this curriculum:

| Concept | Where it appears later |
|---------|------------------------|
| **Tensors** | Embeddings (Week 2) are vectors; attention matrices are 4D tensors |
| **Broadcasting** | Normalization in transformers; batch operations everywhere |
| **Indexing** | Gathering embeddings; masking in attention |
| **Autograd** | Training any model; computing Jacobians (Week 5-6) |
| **GPU** | Practical necessity for any non-trivial model |

The gradient computation you just did by hand? That's exactly what happens trillions of times when training GPT-4. Autograd makes it tractable.

---

# Resources

**Official Documentation**:
- [PyTorch Tensors Tutorial](https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html)
- [NumPy Broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html)
- [PyTorch Autograd](https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html)

**Deep Dives**:
- [Jay Alammar: A Visual Intro to NumPy](https://jalammar.github.io/visual-numpy/)
- [PyTorch Internals: Autograd](http://blog.ezyang.com/2019/05/pytorch-internals/)

**Wiki**:
- [Glossary](../../wiki/glossary.md) — tensor, broadcasting, autograd

---

# Reflection Questions

Before moving on, consider:

1. **Why do we need autograd instead of computing gradients by hand?**
   - Think about a neural network with millions of parameters...

2. **When would broadcasting cause unexpected behavior?**
   - What if you accidentally have a (3,) vector and a (3, 1) vector?

3. **Why is GPU memory transfer a bottleneck?**
   - What does this imply for how we should structure data pipelines?

---

# Next Up

In **01b: Linear Algebra Refresh**, we'll revisit:
- Vectors as arrows, matrices as transformations
- Eigenvalues and eigenvectors (crucial for understanding dynamics)
- SVD for dimensionality reduction

All using the tensor tools you just learned.

→ [01b: Linear Algebra Refresh](01b-linear-algebra-refresh.ipynb)